In [ ]:
!pip install pytrends
from pytrends.request import TrendReq
import pandas as pd
from datetime import date
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe

In [ ]:
## daily range
## sample date range from 2020-02-01 to 2020-03-01
date_range = pd.date_range(start="2020-12-01", end='2021-01-01')

In [ ]:
## import pytrends, final df format
pytrends = TrendReq(hl='en-US', tz=360)
final_df = pd.DataFrame([], columns = ['geoName', 'geoCode', 'topic', 'date'])

In [ ]:
## keywords
## sample keywords = housing, unemployment
kw_list = ['Covid-19 + Pandemic + Quarantine + Covid + Coronavirus']
cat_list = ['Covid-19']

In [ ]:
idx = 0

for i in kw_list:
  ## temporary df format for each topic
  new_df = pd.DataFrame([], columns = ['geoName', 'geoCode', i, 'date'])
  
  for j in range(0, len(date_range)-1):
    tf = '' + date_range[j].strftime('%Y-%m-%d') + ' ' + date_range[j+1].strftime('%Y-%m-%d')
    pytrends.build_payload([i], timeframe = tf, geo='US')
    df = pytrends.interest_by_region(resolution='COUNTY', inc_low_vol=True, inc_geo_code=True)
    df = df.reset_index()

    date = []
    for k in range(0, len(df)):
      date.append(date_range[j].strftime('%Y-%m-%d'))
        
    df['date'] = date
    new_df = new_df.append(df, ignore_index=True)
  

  new_df.rename(columns={i: 'value'}, inplace = True)

  ## add topic column
  cat = cat_list[idx]
  idx += 1
  topic = []
  for l in range(0, len(new_df)):
    topic.append(cat)
  new_df['topic'] = topic
  
  final_df = final_df.append(new_df, ignore_index= True)

In [ ]:
## extract GA
GA = final_df[final_df['geoCode']=='US-GA']

## rename geoname of final_data to US
final_df.geoName = 'US'
final_df.geoCode = 'US'

## merge two
final_df = final_df.append(GA, ignore_index= True)

In [ ]:
final_df

,geoName,geoCode,topic,date,value
0,US,US,Covid-19,2020-12-01,34
1,US,US,Covid-19,2020-12-01,73
2,US,US,Covid-19,2020-12-01,73
3,US,US,Covid-19,2020-12-01,46
4,US,US,Covid-19,2020-12-01,64
...,...,...,...,...,...
1607,Georgia,US-GA,Covid-19,2020-12-27,71
1608,Georgia,US-GA,Covid-19,2020-12-28,77
1609,Georgia,US-GA,Covid-19,2020-12-29,72
1610,Georgia,US-GA,Covid-19,2020-12-30,63


In [ ]:
len(final_df['topic'].unique()) ## confirm that there are 17 topics

In [ ]:
## connecting gspreads
scope = [
'https://spreadsheets.google.com/feeds'
]
json_file_name = '/directory_stored_json_file/***.json' ## refer to the README to download your own JSON file
creds = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
client = gspread.authorize(creds)

## open by url
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1mOADAGBLW8m7s2ULWih0vRZtsLUn7VFzRMaLVnieFu8/' # go on link to view dataset
sheet = client.open_by_url(spreadsheet_url)
worksheet = sheet.worksheet('Sheet1')

## get existing dataframe
existing = get_as_dataframe(worksheet)
existing = existing.dropna(how="all")
existing = existing.iloc[:, 0:5]

In [ ]:
## update existing dataframe
update = existing.append(final_df, ignore_index= True)
update

In [ ]:
## update google sheet
set_with_dataframe(worksheet, update)